In [2]:
# ================================
# 1. Imports & basic configuration
# ================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import scipy

from scipy.ndimage import median_filter
from scipy.signal import find_peaks

plt.style.use("default")  # keep plots clean

# Path to the Dips + Pull-ups CSV file (relative to this notebook)
DATA_PATH = Path("..") / "data" / "Push_Ups.csv"

print("Using data file:", DATA_PATH)

# We only analyse the first 90 seconds of recording
MAX_DURATION_S = 88.0

# Parameters for ENMO smoothing
MEDIAN_FILTER_SIZE = 5          # in samples
MOVING_AVERAGE_WINDOW_S = 0.2   # seconds

# Repetition detection parameters for push-ups
PUSHUPS_MIN_INTERVAL_S = 1.0    # minimal time between two push-ups (s)
PUSHUPS_PERCENTILE = 85         # percentile used for threshold on ENMO_smooth


c:\Users\Hp\miniconda3\Lib\site-packages\matplotlib\projections\__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


Using data file: ..\data\Push_Ups.csv


In [3]:
# =======================================
# 2. Load raw accelerometer data from CSV
# =======================================

def load_pushups_csv(path: str) -> pd.DataFrame:
    """
    Load push-ups accelerometer data exported from OmGui.

    Assumes CSV has no header and columns are:
        0: timestamp (string)
        1: Ax
        2: Ay
        3: Az

    Returns a DataFrame with:
        - timestamp (datetime)
        - t (seconds from start)
        - Ax, Ay, Az (float)
    """
    df = pd.read_csv(path, header=None)
    df.columns = ["timestamp", "Ax", "Ay", "Az"]

    df["timestamp"] = pd.to_datetime(df["timestamp"])
    t0 = df["timestamp"].iloc[0]
    df["t"] = (df["timestamp"] - t0).dt.total_seconds()

    return df


df_raw = load_pushups_csv(DATA_PATH)

print("Number of samples:", len(df_raw))
print("Recording duration (s):", df_raw["t"].iloc[-1])
df_raw.head()

Number of samples: 21000
Recording duration (s): 205.371


,timestamp,Ax,Ay,Az,t
0,2025-11-27 15:00:06.779,0.125000,-0.203125,-1.031250,0.000
1,2025-11-27 15:00:06.789,-0.031250,-0.375000,1.312500,0.010
2,2025-11-27 15:00:06.800,0.031250,-0.390625,1.234375,0.021
3,2025-11-27 15:00:06.809,0.078125,-0.390625,1.093750,0.030
4,2025-11-27 15:00:06.819,0.046875,-0.390625,1.125000,0.040
